# Rhyme

In [1]:
# !pip install -r ../requirements.txt
import sys
sys.path.append('../')
from generative_formalism import *

In [2]:
df = get_rhyme_for_completed_poems()
df.line_sim.max()

100%|██████████| 22458/22458 [00:04<00:00, 5173.09it/s]


np.float64(94.73684210526316)

In [3]:
df[df.model!=HIST].id_human.nunique()

5823

In [4]:
df[df.model!=HIST].period.value_counts()

period
1700-1750    1283
1650-1700    1223
1900-1950    1149
1600-1650    1135
1950-2000    1100
1750-1800     965
1850-1900     958
1800-1850     758
Name: count, dtype: int64

In [5]:
len(df[df.model!=HIST])

8571

In [6]:
df.model9.value_counts()

model9
                   6269
Llama3.1           3087
Olmo2              2533
Deepseek           1153
Claude-3-Sonnet     987
GPT-3.5-Turbo       811
Name: count, dtype: int64

In [14]:
df_prompts = df.copy().query('model!="(Historical)"')

df_prompts_stats = pd.DataFrame([
    {
        'Period':period,
        'Model':model,
        '# Poems': len(gdf),
        # '\# Poems per model': int(round(len(gdf) / gdf.model.nunique())),
        }
    for (period,model),gdf in df_prompts.groupby(['period','model9'])
])

df_prompts_stats = df_prompts_stats.pivot_table(index='Period', columns='Model', values='# Poems', aggfunc='sum')
df_prompts_stats.to_latex('table_completions_nums.tex')
df_prompts_stats

Model,Claude-3-Sonnet,Deepseek,GPT-3.5-Turbo,Llama3.1,Olmo2
Period,,,,,
1600-1650,145,187,95,403,305
1650-1700,126,182,105,466,344
1700-1750,152,175,151,488,317
1750-1800,117,122,116,330,280
1800-1850,84,88,63,264,259
1850-1900,101,103,78,364,312
1900-1950,140,148,96,395,370
1950-2000,122,148,107,377,346


In [22]:
df_prompts.model.unique()

array(['Olmo', '(Historical)', 'Llama', 'ChatGPT', 'Claude', 'DeepSeek'],
      dtype=object)

In [6]:
figdf_avg = get_avgs_df(df, gby=['period','model'])
figdf_avg['count'].describe()

count     48.000000
mean     309.166667
std      244.869338
min       63.000000
25%      120.750000
50%      223.000000
75%      381.500000
max      944.000000
Name: count, dtype: float64

In [7]:
figdf_avg[figdf_avg.model!=HIST]['count'].describe()

count     40.000000
mean     214.275000
std      122.988219
min       63.000000
25%      113.750000
50%      151.500000
75%      320.250000
max      488.000000
Name: count, dtype: float64

In [8]:
figdf_avg[figdf_avg.model!=HIST]['count'].sum()

np.int64(8571)

In [9]:
p9.options.figure_size = (10, 6)
models = [HIST] + [x for x in sorted(figdf_avg.model.unique()) if x!=HIST]
figdf_avg['model'] = pd.Categorical(figdf_avg.model, categories=models)

fig = p9.ggplot(figdf_avg, p9.aes(x='period', y='mean', color='model', group='model'))
fig += p9.geom_point(p9.aes(size='count'), shape='o', alpha=.3)
fig += p9.geom_line()
fig += p9.geom_errorbar(
    p9.aes(ymin='mean - stderr', ymax='mean + stderr'),
    width=0.25,
    # alpha=0.6
)
fig += p9.theme_minimal()

fig += p9.geom_text(
    p9.aes(label='model'),
    data=figdf_avg[figdf_avg.period=='1950-2000'],
    adjust_text={'x':0.05, 'y':0.05, 'arrowprops': dict(arrowstyle='-', alpha=0)},
    
    show_legend=False,
)

fig += p9.theme(panel_background=p9.element_rect(fill='white'), plot_background=p9.element_rect(fill='white'))
fig += p9.geom_hline(yintercept=50, color='gray', linetype='dashed')
fig += p9.scale_y_continuous(limits=(0,100))
fig += p9.labs(
    x='Half-century of poet\'s birth',
    y='Predicted percentage of rhyming poems',
    color='Model',
    size='Number of poems',
    title='Predicted percentage of rhyming poems in generative completions of poems by their historical period',

)
# fig.save('../figures/rhyme_avgs_by_period_completions2.png')
fig

ModuleNotFoundError: No module named 'adjustText'

In [15]:
df['num_lines'].describe()

count    14813.000000
mean        12.564707
std          3.089484
min         10.000000
25%         11.000000
50%         12.000000
75%         14.000000
max        108.000000
Name: num_lines, dtype: float64

In [ ]:
df_stat_signif = compute_all_stat_signif(df).reset_index()
df_stat_signif[df_stat_signif.comparison.str.contains(HIST)].query('groupby=="1950-2000"').sort_values('effect_size', ascending=False).head(25)

,groupby,comparison,p_value,effect_size,effect_size_str,mean1,mean2,significant
0,1950-2000,(Historical) vs Claude,0.0000,1.461651,large,14.985994,69.672131,True
1,1950-2000,Llama vs (Historical),0.0000,0.997358,large,55.968170,14.985994,True
2,1950-2000,Olmo vs (Historical),0.0000,0.840839,large,49.421965,14.985994,True
5,1950-2000,(Historical) vs DeepSeek,0.0000,0.631730,medium,14.985994,39.189189,True
7,1950-2000,(Historical) vs ChatGPT,0.0001,0.449703,small,14.985994,31.775701,True
